## Causal Inference
- 인과추론의 근본적인 문제
    - 한 개인의 counterfactual은 관측 불가
- selection bias 존재 (선택편향)
    - 처치를 받은 집단과 안받은 집단을 단순비교하면, 원래부터 다른 특성 차이까지 같이 섞여 진짜 효과를 알 수 없음
    - 선택편향 = $E[Y(0)∣T=1]−E[Y(0)∣T=0]$, 처치군이 처치를 받지 않았더라도 평균적으로 통제군과 같았을까?
        - $E[Y(0)∣T=1]>E[Y(0)∣T=0]$일 수 있음 즉, 처치군은 애초에 outcome이 높을 집단 (아래 데이터셋에 의거)
        - RCT: $E[Y(0)∣T=1]=E[Y(0)∣T=0]$ 즉, 선택편향 = 0 &rarr; 단순 평균 차이 = ATE
        - Observational Data: 교란변수로 인해 $E[Y(0)∣T=1] != E[Y(0)∣T=0]$ &rarr; 선택편향 발생
                - 단순평균차이 = 진짜효과 + 원래 집단 차이(선택편향)
                - 인과추론 방법은 **"원래 집단 차이(선택편향)"를 제거하는 것**이 목표

### 1. Potential Outcome Framework

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(42)
n = 5000

# 공변량 (X)
age = np.random.normal(40,10,n)
income = np.random.normal(5000, 1000, n)

# treatment 확률 (confounding존재)
prob_test = 1 / (1 + np.exp(-(0.05*age - 0.0005*income))) # 교란변수 존재를 위해, 교란변수 없으려면 prob_test= 0.5
treatment = np.random.binomial(1, prob_test)

# 진짜 인과효과
true_effect = 5

outcome = (
    0.3*age
    + 0.01*income
    + true_effect*treatment
    + np.random.normal(0,5,n)
)

df = pd.DataFrame({
    "age": age,
    "income": income,
    "treatment": treatment,
    "outcome": outcome
})

In [3]:
df.head()

,age,income,treatment,outcome
0,44.967142,4576.240318,0,72.932346
1,38.617357,4546.585892,0,59.894914
2,46.476885,3204.356827,1,47.837203
3,55.230299,4669.909808,1,70.007768
4,37.658466,5732.829082,0,77.529024


### 2. Confounding 확인
- treatment, control 집단의 평균 나이 및 수입이 다름 -> 교란 존재

In [4]:
df.groupby('treatment')[['age','income']].mean()

,age,income
treatment,,
0,38.253704,5181.255804
1,42.766261,4702.713739


### 3. Regression Adjustment
- Ignorability 가정: (Y(1),Y(0))⊥T∣X
    - X가 주어지면, treatment배정은 잠재결과와 독립이다. 즉, 같은 X를 가진 사람들끼리는 treatment가 무작위처럼 배정됨
    - **숨겨진 교란**이 없다는 가정하에 가능
- 해당 가정이 성립해야하는 이유
    - 해당 데이터는 관측 데이터이기 때문에 개인의 Y(1), Y(0)을 동시에 볼 수 없음
- 핵심 : confounder(교란변수)를 통제하여 treatment의 순수 효과만 분리하는 방법, 즉 x를 고정하고 t만 바꾸자
    - e.g. age와 income을 고정했을 때 treatment가 outcome을 얼마나 변화시키는가?
- 수학적 구조
    - outcome 생성식 : Y=0.3age+0.01income+5T+ϵ
    - 우리가 추정하고자 하는 모형 : Y=αT+β1age+β2income+ϵ / 이때 α가 ATE 추정치
    - OLS : 해당 식에서 실제 Y값(outcome)과 예측 Y값의 오차가 가장 최소가 되도록 하는 α, β1, β2값을 찾자

In [7]:
import statsmodels.api as sm

X = df[['treatment','age','income']]
X = sm.add_constant(X)

model = sm.OLS(df['outcome'], X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     7470.
Date:                Fri, 20 Feb 2026   Prob (F-statistic):               0.00
Time:                        11:31:07   Log-Likelihood:                -15158.
No. Observations:                5000   AIC:                         3.032e+04
Df Residuals:                    4996   BIC:                         3.035e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1539      0.465     -2.482      0.0